# **Object Tracking**

1.  Implement a Kalman filter to predict and update the state of an object given its measurements


In [1]:
import numpy as np

class KalmanFilter:
    def __init__(self, dt, process_noise, measurement_noise):
        # State vector: [position, velocity]
        self.x = np.array([[0], [0]])  # Initial state (position=0, velocity=0)

        # State covariance matrix
        self.P = np.eye(2)

        # State transition matrix
        self.F = np.array([[1, dt],  # dt is the time step
                           [0, 1]])

        # Process noise covariance matrix
        self.Q = process_noise * np.array([[dt**4/4, dt**3/2],
                                           [dt**3/2, dt**2]])

        # Measurement matrix
        self.H = np.array([[1, 0]])  # We only observe position

        # Measurement noise covariance matrix
        self.R = np.array([[measurement_noise]])

        # Identity matrix
        self.I = np.eye(2)

    def predict(self):
        # Predict the state
        self.x = np.dot(self.F, self.x)

        # Predict the state covariance
        self.P = np.dot(self.F, np.dot(self.P, self.F.T)) + self.Q

    def update(self, z):
        # Measurement residual
        y = z - np.dot(self.H, self.x)

        # Residual covariance
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R

        # Kalman gain
        K = np.dot(self.P, np.dot(self.H.T, np.linalg.inv(S)))

        # Update the state
        self.x = self.x + np.dot(K, y)

        # Update the covariance
        self.P = np.dot((self.I - np.dot(K, self.H)), self.P)

    def get_state(self):
        return self.x

# Parameters
dt = 1  # Time step
process_noise = 1e-5  # Process noise
measurement_noise = 1  # Measurement noise

# Create a Kalman filter instance
kf = KalmanFilter(dt, process_noise, measurement_noise)

# Simulated measurements (positions)
measurements = [1, 2, 3, 4, 5, 6, 7, 8]

# Predict and update the Kalman filter for each measurement
for z in measurements:
    kf.predict()  # Predict step
    kf.update(np.array([[z]]))  # Update step with the new measurement
    state = kf.get_state()
    print(f"Predicted Position: {state[0, 0]}, Velocity: {state[1, 0]}")


Predicted Position: 0.666666944444213, Velocity: 0.33333472222106486
Predicted Position: 1.6666686111016205, Velocity: 0.6666705555356484
Predicted Position: 2.750004010370302, Velocity: 0.833338923540879
Predicted Position: 3.818188119695462, Velocity: 0.9090978097474491
Predicted Position: 4.864873894217582, Velocity: 0.9459540664794636
Predicted Position: 5.896564036052852, Velocity: 0.9655265748121687
Predicted Position: 6.918620848074862, Velocity: 0.9767547431965324
Predicted Position: 7.934446931591791, Velocity: 0.9836183504245745


2. Write a function to normalize an image array such that pixel values are scaled between 0 and 1

In [2]:
import numpy as np

def normalize_image(image):
    """
    Normalize an image array to scale pixel values between 0 and 1.

    Parameters:
        image (numpy.ndarray): Input image array with pixel values.

    Returns:
        numpy.ndarray: Normalized image array with values in the range [0, 1].
    """
    # Ensure the image is in float format to avoid truncation during division
    image = image.astype(np.float32)

    # Find the minimum and maximum pixel values
    min_val = np.min(image)
    max_val = np.max(image)

    # Normalize the image
    normalized_image = (image - min_val) / (max_val - min_val)

    return normalized_image


In [3]:
# Example image array
image = np.array([[50, 100, 150],
                  [200, 250, 255],
                  [0, 75, 125]], dtype=np.uint8)

# Normalize the image
normalized_image = normalize_image(image)

# Print the result
print("Original Image:\n", image)
print("Normalized Image:\n", normalized_image)


Original Image:
 [[ 50 100 150]
 [200 250 255]
 [  0  75 125]]
Normalized Image:
 [[0.19607843 0.39215687 0.5882353 ]
 [0.78431374 0.98039216 1.        ]
 [0.         0.29411766 0.49019608]]


3.  Create a function to generate dummy object detection data with confidence scores and bounding boxes.
Filter the detections based on a confidence threshold

In [4]:
import numpy as np

def generate_dummy_detections(num_detections=10):
    """
    Generate dummy object detection data with confidence scores and bounding boxes.

    Parameters:
        num_detections (int): Number of dummy detections to generate.

    Returns:
        list: A list of dictionaries, where each dictionary represents a detection
              with a bounding box and a confidence score.
    """
    dummy_detections = []
    for _ in range(num_detections):
        detection = {
            "bounding_box": [
                np.random.randint(0, 100),  # x_min
                np.random.randint(0, 100),  # y_min
                np.random.randint(100, 200),  # x_max
                np.random.randint(100, 200)   # y_max
            ],
            "confidence": np.random.uniform(0.0, 1.0)  # Random confidence score
        }
        dummy_detections.append(detection)
    return dummy_detections

def filter_detections(detections, confidence_threshold=0.5):
    """
    Filter detections based on a confidence threshold.

    Parameters:
        detections (list): A list of detection dictionaries with bounding boxes and confidence scores.
        confidence_threshold (float): The threshold for filtering detections.

    Returns:
        list: A list of filtered detections.
    """
    filtered_detections = [
        detection for detection in detections
        if detection["confidence"] >= confidence_threshold
    ]
    return filtered_detections


In [5]:
# Generate dummy detections
detections = generate_dummy_detections(num_detections=10)

# Set confidence threshold
confidence_threshold = 0.6

# Filter detections
filtered_detections = filter_detections(detections, confidence_threshold)

# Display results
print("Original Detections:")
for det in detections:
    print(det)

print("\nFiltered Detections:")
for det in filtered_detections:
    print(det)


Original Detections:
{'bounding_box': [63, 19, 168, 190], 'confidence': 0.10752482490184534}
{'bounding_box': [66, 32, 178, 122], 'confidence': 0.7370863995279016}
{'bounding_box': [26, 37, 173, 144], 'confidence': 0.5466456157309278}
{'bounding_box': [48, 92, 101, 130], 'confidence': 0.06858801561116201}
{'bounding_box': [86, 93, 109, 115], 'confidence': 0.41038712752187145}
{'bounding_box': [54, 44, 176, 133], 'confidence': 0.6586873270968704}
{'bounding_box': [66, 65, 185, 102], 'confidence': 0.44116299630636213}
{'bounding_box': [37, 25, 170, 120], 'confidence': 0.03826876573552895}
{'bounding_box': [77, 82, 182, 164], 'confidence': 0.598034281995006}
{'bounding_box': [4, 59, 104, 122], 'confidence': 0.8470328190176433}

Filtered Detections:
{'bounding_box': [66, 32, 178, 122], 'confidence': 0.7370863995279016}
{'bounding_box': [54, 44, 176, 133], 'confidence': 0.6586873270968704}
{'bounding_box': [4, 59, 104, 122], 'confidence': 0.8470328190176433}


4.  Write a function that takes a list of YOLO detections and extracts a random 128-dimensional feature vector
for each detection

In [6]:
import numpy as np

def extract_features_from_yolo_detections(yolo_detections):
    """
    Takes a list of YOLO detections and extracts a random 128-dimensional feature vector for each detection.

    Parameters:
        yolo_detections (list): A list of dictionaries, where each dictionary contains a bounding box
                                and a confidence score for the detection.
                                Example format:
                                [
                                    {"bounding_box": [x_min, y_min, x_max, y_max], "confidence": 0.85},
                                    ...
                                ]

    Returns:
        list: A list of dictionaries, where each dictionary contains the original detection data
              and a 128-dimensional feature vector.
    """
    features = []
    for detection in yolo_detections:
        feature_vector = np.random.rand(128)  # Generate random 128-dimensional feature vector
        detection_with_features = {
            "bounding_box": detection["bounding_box"],
            "confidence": detection["confidence"],
            "feature_vector": feature_vector
        }
        features.append(detection_with_features)
    return features


In [7]:
# Example YOLO detections
yolo_detections = [
    {"bounding_box": [34, 58, 120, 170], "confidence": 0.92},
    {"bounding_box": [10, 20, 50, 80], "confidence": 0.76},
    {"bounding_box": [60, 40, 150, 190], "confidence": 0.85},
]

# Extract features
detections_with_features = extract_features_from_yolo_detections(yolo_detections)

# Display results
for detection in detections_with_features:
    print(f"Bounding Box: {detection['bounding_box']}")
    print(f"Confidence: {detection['confidence']}")
    print(f"Feature Vector (first 5 values): {detection['feature_vector'][:5]}...\n")


Bounding Box: [34, 58, 120, 170]
Confidence: 0.92
Feature Vector (first 5 values): [0.94255317 0.48709673 0.3234599  0.41207027 0.14307764]...

Bounding Box: [10, 20, 50, 80]
Confidence: 0.76
Feature Vector (first 5 values): [0.60680957 0.7020828  0.09200547 0.14631727 0.14685044]...

Bounding Box: [60, 40, 150, 190]
Confidence: 0.85
Feature Vector (first 5 values): [0.72995552 0.9506403  0.48880731 0.67735397 0.81778207]...



5. Write a function to re-identify objects by matching feature vectors based on Euclidean distance

In [8]:
import numpy as np

def reidentify_objects(feature_vectors_query, feature_vectors_database, threshold=0.5):
    """
    Re-identify objects by matching feature vectors from a query set with a database
    based on Euclidean distance.

    Parameters:
        feature_vectors_query (list): List of feature vectors for query objects.
                                      Example: [np.array([...]), np.array([...]), ...]
        feature_vectors_database (list): List of feature vectors in the database.
                                          Example: [np.array([...]), np.array([...]), ...]
        threshold (float): Maximum distance below which two vectors are considered a match.

    Returns:
        list: A list of matches for each query. Each match is represented as a dictionary:
              {
                  "query_index": int,           # Index of the query vector
                  "matched_database_index": int,  # Index of the matched database vector
                  "distance": float              # Distance between the two vectors
              }
    """
    matches = []
    for i, query_vector in enumerate(feature_vectors_query):
        min_distance = float("inf")
        best_match_index = -1

        for j, db_vector in enumerate(feature_vectors_database):
            # Compute Euclidean distance
            distance = np.linalg.norm(query_vector - db_vector)

            # Update if this is the closest match so far
            if distance < min_distance:
                min_distance = distance
                best_match_index = j

        # Check if the match is within the acceptable threshold
        if min_distance <= threshold:
            matches.append({
                "query_index": i,
                "matched_database_index": best_match_index,
                "distance": min_distance
            })
    return matches


In [9]:
# Example feature vectors
feature_vectors_query = [
    np.random.rand(128),  # Random query feature vector
    np.random.rand(128)
]

feature_vectors_database = [
    np.random.rand(128),  # Random database feature vector
    np.random.rand(128),
    np.random.rand(128)
]

# Re-identify objects
matches = reidentify_objects(feature_vectors_query, feature_vectors_database, threshold=10.0)

# Display matches
for match in matches:
    print(f"Query Index: {match['query_index']} matched with Database Index: {match['matched_database_index']}")
    print(f"Distance: {match['distance']:.4f}\n")


Query Index: 0 matched with Database Index: 2
Distance: 4.6927

Query Index: 1 matched with Database Index: 1
Distance: 4.6674



6. Write a function to track object positions using YOLO detections and a Kalman Filter

In [10]:
import numpy as np

class KalmanFilter:
    def __init__(self):
        # State [x, y, dx, dy] (x, y are position; dx, dy are velocity)
        self.state = np.zeros(4)

        # State covariance matrix
        self.P = np.eye(4) * 1000

        # Transition matrix (assuming constant velocity model)
        self.F = np.array([[1, 0, 1, 0],
                           [0, 1, 0, 1],
                           [0, 0, 1, 0],
                           [0, 0, 0, 1]])

        # Measurement matrix
        self.H = np.array([[1, 0, 0, 0],
                           [0, 1, 0, 0]])

        # Measurement noise covariance
        self.R = np.array([[10, 0],
                           [0, 10]])

        # Process noise covariance
        self.Q = np.array([[1, 0, 0, 0],
                           [0, 1, 0, 0],
                           [0, 0, 1, 0],
                           [0, 0, 0, 1]])

    def predict(self):
        # Predict the next state
        self.state = np.dot(self.F, self.state)
        self.P = np.dot(self.F, np.dot(self.P, self.F.T)) + self.Q

        return self.state

    def update(self, measurement):
        # Kalman Gain
        y = measurement - np.dot(self.H, self.state)
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R
        K = np.dot(self.P, np.dot(self.H.T, np.linalg.inv(S)))

        # Update the state
        self.state = self.state + np.dot(K, y)
        self.P = self.P - np.dot(K, np.dot(self.H, self.P))

        return self.state


In [11]:
import cv2
import numpy as np

def track_objects_with_kalman(yolo_detections, kalman_filters, threshold=0.5):
    """
    Track objects using YOLO detections and Kalman Filter.

    Parameters:
        yolo_detections (list): List of bounding boxes from YOLO detections.
                                 Example: [(x1, y1, x2, y2), ...]
        kalman_filters (list): List of Kalman Filter objects used to track the objects.
        threshold (float): Threshold for distance to consider a detection as a valid match.

    Returns:
        tracked_positions (list): List of updated object positions [(x, y), ...]
    """
    tracked_positions = []

    # For each object in YOLO detections, perform Kalman Filter update
    for i, detection in enumerate(yolo_detections):
        # Get the center of the bounding box (x, y)
        x_center = (detection[0] + detection[2]) / 2
        y_center = (detection[1] + detection[3]) / 2

        if i < len(kalman_filters):
            # If the Kalman filter already exists for this object, update it
            kf = kalman_filters[i]
            prediction = kf.predict()
            predicted_position = prediction[:2]  # predicted (x, y)

            # Compute the Euclidean distance between predicted and detection
            distance = np.linalg.norm(np.array([x_center, y_center]) - predicted_position)

            if distance < threshold:
                # Update the Kalman filter with the new detection if the distance is small
                kf.update(np.array([x_center, y_center]))
            else:
                # Reset the Kalman filter for a new object if the detection is far away
                kf = KalmanFilter()
                kf.update(np.array([x_center, y_center]))
                kalman_filters.append(kf)

            # Get the updated position from the Kalman filter
            updated_position = kf.state[:2]
            tracked_positions.append(updated_position)
        else:
            # If there is no Kalman filter for this object, create one
            kf = KalmanFilter()
            kf.update(np.array([x_center, y_center]))
            kalman_filters.append(kf)

            # Get the updated position from the Kalman filter
            updated_position = kf.state[:2]
            tracked_positions.append(updated_position)

    return tracked_positions


In [12]:
# Simulating YOLO detections: list of bounding boxes (x1, y1, x2, y2)
yolo_detections = [
    (100, 150, 200, 250),  # Detection 1
    (300, 350, 400, 450),  # Detection 2
    (500, 550, 600, 650)   # Detection 3
]

# Create Kalman filters for tracking (initially empty)
kalman_filters = []

# Track the objects across frames
tracked_positions = track_objects_with_kalman(yolo_detections, kalman_filters)

# Display the tracked positions
for i, position in enumerate(tracked_positions):
    print(f"Object {i+1} tracked position: {position}")


Object 1 tracked position: [148.51485149 198.01980198]
Object 2 tracked position: [346.53465347 396.03960396]
Object 3 tracked position: [544.55445545 594.05940594]
